In [0]:
from pyspark.ml import PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql import functions as F

# Load slicing stage (top-10 features)
slicer_model = PipelineModel.load("/FileStore/models/slicer_top10")

# Load preprocessed datasets
train_ready = spark.read.format("delta").load("/FileStore/data/train_ready")
val_ready   = spark.read.format("delta").load("/FileStore/data/val_ready")

# Apply slicer to keep only top-10 features
train_topk = slicer_model.transform(train_ready)
val_topk   = slicer_model.transform(val_ready)

# Efficient class balancing using sampleBy
fractions = {0: 0.6, 1: 1.0}  # 60% of majority, 100% of minority
train_sample = train_topk.sampleBy("label", fractions=fractions, seed=42)

# Define Decision Tree
dt = DecisionTreeClassifier(
    labelCol="label",
    featuresCol="features_topK",
    seed=42
)

# Small param grid (to limit resource usage)
param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [32, 64]) \
    .build()

# Evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

# TrainValidationSplit (full training set used for internal tuning)
tvs = TrainValidationSplit(
    estimator=dt,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    trainRatio=1.0,
    parallelism=1  # For Databricks CE
)

# Train the model with grid search
tvs_model = tvs.fit(train_sample)

# Evaluate on external validation set
val_preds = tvs_model.transform(val_topk)
f1_score = evaluator.evaluate(val_preds)

print(f"Best Decision Tree model F1-score on validation set: {f1_score:.4f}")

# Save best model
tvs_model.bestModel.write().overwrite().save("/FileStore/models/dt_top10_model_grid")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2282065208578701>:55
     46 tvs = TrainValidationSplit(
     47     estimator=lr,
     48     estimatorParamMaps=param_grid,
   (...)
     51     parallelism=1
     52 )
     54 # 11. Treino
---> 55 tvs_model = tvs.fit(train_sample)
     57 # 12. Avaliação
     58 val_preds = tvs_model.transform(val_topk)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/spark/python/pyspark/ml/base.py:205, in Estimator.fit(self, dataset, params)
    203         return self.copy(params)._fit(dataset)
    204     else:
--> 205         return self._fit(data